In [1]:
# import Pkg; Pkg.add("Tables"); Pkg.add("MarkdownTables"); Pkg.add("DataFrames")

In [3]:
using Tables
using MarkdownTables
# using ProgressMeter
include("datasetloader.jl")
include("iso639code.jl")

normcode (generic function with 2 methods)

In [4]:
TLANGS = ["ara", "bel", "ben", "bul", "cat", "ces", "dan", "deu", "ell", "eng", "epo", "pes", "fin",
 "fra", "hau", "srp", "heb", "hin", "hun", "ido", "ina", "isl", "ita", "jpn", "kab", "kor", "ckb", 
 "lat", "lit", "mar", "mkd", "ind", "nds", "nld", "nob", "pol", "por", "ron", "rus", "slk", "spa", 
 "swc", "swe", "tat", "tgl", "tur", "ukr", "vie", "yid", "cmn"]
WLANGS = ["ar", "be", "bn", "bg", "ca", "cs", "da", "de", "el", 
"en", "eo", "fa", "fi", "fr", "ha", "sr", "he", "hi", "hu", "io", 
"ia", "is", "it", "ja", "kab", "ko", "ckb", "la", "lt", "mr", "mk",
 "id", "nds", "nl", "no", "pl", "pt", "ro", "ru", "sk", "es", "sw",
  "sv", "tt", "tl", "tr", "uk", "vi", "yi", "zh"]
LANGS = ["ara", "bel", "ben", "bul", "cat", "ces", "dan", "deu", "ell", 
"eng", "epo", "fas", "fin", "fra", "hau", "hbs", "heb", "hin", "hun", 
"ido", "ina", "isl", "ita", "jpn", "kab", "kor", "kur", "lat", "lit", 
"mar", "mkd", "msa", "nds", "nld", "nor", "pol", "por", "ron", "rus", 
"slk", "spa", "swa", "swe", "tat", "tgl", "tur", "ukr", "vie", "yid", 
"zho"]
WV = WikiDataSet("corpus/wikipedia/test", langs=WLANGS)
TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=TLANGS)

521469-element TatoebaDataset:
             "Pardonu, permesu al mi indiki tri erarojn en la supra artikolo." => "epo"
                                          "Mi veturigos vin al la flughaveno." => "epo"
                                                                 "Kio mankas?" => "epo"
                                            "Aĥ, kiam ili renkontiĝos denove?" => "epo"
 "Kiam mi estis infano, mi pasigis multajn horojn legante sola en mia ĉambro." => "epo"
                                              "Ŝajnas, ke la trajno malfruas." => "epo"
                                                                "Venu rapide!" => "epo"
                                                   "Li montris al mi albumon." => "epo"
                                                             "Estis du kukoj." => "epo"
                                      "Mia filo scipovas jam nombri ĝis cent." => "epo"
                                                                               ⋮
        

In [5]:
WV

10000-element WikiDataSet:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [ ]:
# import Pkg; Pkg.add(["LanguageFinder", "LanguageDetect", "Languages"])

In [7]:
# import LanguageFinder: LanguageFind
# detector_lf(t) = LanguageFind(t, 0).lang
import LanguageDetect: detect
detector_ld(t) = first(detect(t)).language
import Languages: LanguageDetector, isocode
Languages_detector = LanguageDetector()
function detector_ls(t)
    l = Languages_detector(t) |> first
    if l == nothing
        return ""
    else
     return l |> isocode
    end
end
using Logging
Logging.disable_logging(Logging.Info)

LogLevel(1)

In [8]:
function bmk(detector, D, langs)
    tp = zeros(length(langs))
    total = zeros(length(langs))
    for (x, y) in D
        y = normcode(y)
        if y in langs
            ind = findfirst(isequal(y), langs)
            y_ = ""
            try
                y_ = normcode(detector(x))
            catch
                println("Error: $detector($(repr(x)))")
            end
            if y_ == y
                tp[ind] += 1
            end
            total[ind] += 1
        end
    end
    tp ./ total
end


bmk (generic function with 1 method)

In [10]:
using Printf
function showtable(rows...; header=true)
    fmt(v) = v isa Real ? (v!=0 ? (@sprintf "%0.2f%%" v*100) : "-") : v
    tb = hcat((r->vcat(r...)).(rows)...)
    hd = (header isa Bool) ? header : [""; header]
    markdown_table(Tables.table(permutedims(tb), header=hd),String, formatter=fmt)|>print
    markdown_table(Tables.table(permutedims(tb), header=hd), formatter=fmt)
end

showtable (generic function with 1 method)

## tatoeba

In [20]:
@time bmk_ls = bmk(detector_ls, TV, LANGS);
@time bmk_ld = bmk(detector_ld, TV, LANGS);
# @time bmk_lf = bmk(detector_lf, TV, LANGS);
;

Error: detector_ls("w!")


Error: detector_ls("Muiriel은 지금 20살입니다.")
Error: detector_ls("비밀번호는 \"Muiriel\" 입니다.")
Error: detector_ls("1, 3 그리고 5는 홀수이다.")
Error: detector_ls("Muiriel은 지금 20살이야.")
Error: detector_ls("냠!")
Error: detector_ls("응.")


Error: detector_ls("Se alkaa 6:30.")


Error: detector_ls("Hän syntyi 28.7.1888.")


Error: detector_ls("Öö...")
Error: detector_ls("Yäk!")


...

Error: detector_ls("Ồ, OK.")


Error: detector_ls("Пиев.")


Error: detector_ls("Mlč!")


319.487865 seconds (139.70 M allocations: 8.376 GiB, 0.41% gc time, 0.19% compilation time)


1466.109941 seconds (10.92 G allocations: 360.185 GiB, 2.63% gc time, 0.01% compilation time)


In [61]:
showtable("Languages.jl"=>bmk_ls, "LanguageDetect.jl"=>bmk_ld, header=LANGS)

|                   | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra    | hau    | hbs    | heb     | hin    | hun    | ido | ina | isl | ita    | jpn    | kab | kor    | kur | lat | lit    | mar    | mkd    | msa    | nds | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat | tgl    | tur    | ukr    | vie    | yid    | zho    |
|-------------------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|-----|-----|-----|--------|--------|-----|--------|-----|-----|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|
|      Languages.jl | 85.55% | 80.47% | 100.00% | 62.46% |      - | 48.90% | 47.06% | 90.48% |  99

|                   | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra    | hau    | hbs    | heb     | hin    | hun    | ido | ina | isl | ita    | jpn    | kab | kor    | kur | lat | lit    | mar    | mkd    | msa    | nds | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat | tgl    | tur    | ukr    | vie    | yid    | zho    |
|-------------------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|-----|-----|-----|--------|--------|-----|--------|-----|-----|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|
|      Languages.jl | 85.55% | 80.47% | 100.00% | 62.46% |      - | 48.90% | 47.06% | 90.48% |  99.89% | 78.21% | 64.61% | 95.00% | 76.87% | 82.21% | 92.85% | 60.28% |  95.75% | 62.99% | 73.99% |   - |   - |   - | 66.27% | 99.97% |   - | 98.97% |   - |   - | 61.94% | 72.05% | 51.40% | 71.26% |   - | 78.91% | 66.74% | 72.66% | 77.35% | 70.87% | 52.59% |      - | 61.89% |      - | 52.46% |   - | 63.96% | 52.10% | 62.63% | 84.06% | 98.39% | 99.86% |
| LanguageDetect.jl | 93.48% |      - | 100.00% | 62.86% | 57.21% | 71.26% | 51.56% | 81.81% | 100.00% | 74.64% |      - | 93.83% | 90.24% | 77.13% |      - | 26.90% | 100.00% | 92.91% | 86.44% |   - |   - |   - | 69.22% | 99.85% |   - | 99.48% |   - |   - | 81.41% | 86.98% | 73.93% | 85.34% |   - | 65.77% | 55.50% | 92.64% | 70.23% | 84.87% | 78.17% | 57.37% | 60.48% | 84.98% | 69.76% |   - | 90.72% | 90.39% | 72.41% | 99.75% |      - | 98.64% |


## wikipedia

In [11]:
@time wbmk_ls = bmk(detector_ls, WV, LANGS);
@time wbmk_ld = bmk(detector_ld, WV, LANGS);
# @time bmk_lf = bmk(detector_lf, TV, LANGS);
;

Error: detector_ls("Aktualaĵoj | Eventoj laŭ tago de jaro\n◄ |\n14-a jarcento |\n15-a jarcento\n| 16-a jarcento\n| ►\n◄ |\n1370-aj jaroj |\n1380-aj jaroj |\n1390-aj jaroj |\n1400-aj jaroj\n| 1410-aj jaroj\n| 1420-aj jaroj\n| 1430-aj jaroj\n| ►\n◄◄ |\n◄ |\n1398 |\n1399 |\n1400 |\n1401 |\n1402\n| 1403  \n| 1404  \n| 1405  \n| 1406  \n| ►\n| ►►\nEn la jaro 1402 post Kristo okazis, interalie:\n")
Error: detector_ls("Aktualaĵoj | Eventoj laŭ tago de jaro\n◄ |\n2-a jarcento |\n3-a jarcento\n| 4-a jarcento\n| ►\n◄ |\n170-aj jaroj |\n180-aj jaroj |\n190-aj jaroj |\n200-aj jaroj\n| 210-aj jaroj\n| 220-aj jaroj\n| 230-aj jaroj\n| ►\n◄◄ |\n◄ |\n202 |\n203 |\n204 |\n205 |\n206\n| 207  \n| 208  \n| 209  \n| 210  \n| ►\n| ►►\nEn la jaro 206 post Kristo okazis, interalie:\n")


Error: detector_ls("Cikloheksila nitrato estas organika kombinaĵo kies molekula formulo estas C6H11NO3 rezultanta el interagado de la nitrata acido kaj cikloheksanolo. Ĝi apartenas al la familio de la esteroj kaj estas senkolora brulema likvaĵo iom malstabila, tute solvebla en akvo kaj solvebla en alkoholo kaj etero.\n +  \n  \n    \n      \n        \n          \n            \n              \n                 \n                \n                  \n                    \n                      H\n                      \n                        2\n                      \n                    \n                    S\n                    \n                      O\n                      \n                        4\n                      \n                    \n                  \n                \n                \n              \n            \n            \n          \n        \n      \n    \n    }\\,}}}\n  \n + \n +  \n  \n    \n      \n        \n          \n            \n              \n  

Error: detector_ls("Lango la Historia | Lango la Biografia | Karibuni | Orodha ya Miaka\n◄ |\nKarne ya 3 KK |\nKarne ya 2 KK |\nKarne ya 1 KK |\n►\n◄ | Miaka ya 130 KK | Miaka ya 120 KK | Miaka ya 110 KK | Miaka ya 100 KK |\nMiaka ya 90 KK |\n►\n◄◄ | ◄ | 117 KK | 116 KK | 115 KK | 114 KK |\n113 KK |\n112 KK |\n111 KK |\n► |\n►►\nMakala hii inahusu mwaka 114 KK .\n")
Error: detector_ls("Lango la Historia | Lango la Biografia | Karibuni | Orodha ya Miaka\n◄ |\nKarne ya 12 |\nKarne ya 13\n| Karne ya 14\n| ►\n◄ |\nMiaka ya 1210 |\nMiaka ya 1220 |\nMiaka ya 1230 |\nMiaka ya 1240\n| Miaka ya 1250\n| Miaka ya 1260\n| Miaka ya 1270\n| ►\n◄◄ |\n◄ |\n1243 |\n1244 |\n1245 |\n1246 |\n1247\n| 1248\n| 1249\n| 1250\n| 1251\n| ►\n| ►►\nMakala hii inahusu mwaka 1247 BK .\nbila tarehe\n")
Error: detector_ls("Lango la Historia | Lango la Biografia | Karibuni | Orodha ya Miaka\n◄ |\nKarne ya 14 |\nKarne ya 15\n| Karne ya 16\n| ►\n◄ |\nMiaka ya 1380 |\nMiaka ya 1390 |\nMiaka ya 1400 |\nMiaka ya 1410\n| Mia

Error: detector_ls("Lango la Historia | Lango la Biografia | Karibuni | Orodha ya Miaka\n◄ |\nKarne ya 14 |\nKarne ya 15\n| Karne ya 16\n| ►\n◄ |\nMiaka ya 1460 |\nMiaka ya 1470 |\nMiaka ya 1480 |\nMiaka ya 1490\n| Miaka ya 1500\n| Miaka ya 1510\n| Miaka ya 1520\n| ►\n1490 |\n1491 |\n1492 |\n1493 |\n1494 |\n1495 |\n1496 |\n1497 |\n1498 |\n1499\nMakala hii inahusu miaka 1490 - 1499.\n")
Error: detector_ls("Lango la Historia | Lango la Biografia | Karibuni | Orodha ya Miaka\n◄ |\nKarne ya 4 |\nKarne ya 5\n| Karne ya 6\n| ►\n◄ |\nMiaka ya 430 |\nMiaka ya 440 |\nMiaka ya 450 |\nMiaka ya 460\n| Miaka ya 470\n| Miaka ya 480\n| Miaka ya 490\n| ►\n460 |\n461 |\n462 |\n463 |\n464 |\n465 |\n466 |\n467 |\n468 |\n469\nMakala hii inahusu muongo ya miaka 460 - 469.\n")
Error: detector_ls("Lango la Historia | Lango la Biografia | Karibuni | Orodha ya Miaka\n◄ |\nKarne ya 7 |\nKarne ya 8\n| Karne ya 9\n| ►\n◄ |\nMiaka ya 760 |\nMiaka ya 770 |\nMiaka ya 780 |\nMiaka ya 790\n| Miaka ya 800\n| Miaka ya 8

Error: detector_ls("پۆتانەکان: 26°18′35″N 50°08′38″E  /  26.30972°N 50.14389°E  / 26.30972; 50.14389\nیەکێکە لە زانکۆکانی وڵاتی سعودیە.\nلیستی زانکۆ عەرەبییەکان\n")


...

Error: detector_ld("Pour les articles homonymes, voir Gleize.\nmodifier - modifier le code - modifier WikidataPour les articles homonymes, voir Maurice Gleize.\nMaurice Joseph Louis Gleize, né à Nîmes le 7 janvier 1907 et mort le 20 avril 2003 à Montfermeil, est un résistant et poète français.\nNé en 1907, issu d'une famille ouvrière, Maurice Gleize entre en apprentissage dans une imprimerie dès l'âge de 12 ans. En 1920, âgé de treize ans, il se syndique à la CGT à l'occasion d'un mouvement de grève parmi les ouvriers du livre. Plus tard, il fréquente le soir l'École des beaux-arts de Nîmes, où il apprend le dessin, et le Conservatoire de musique, où il s'initie au violoncelle, qu'il pratique ensuite en orchestre. Marié, père d'un enfant, il exerce son métier d'imprimeur à Paris à partir de 1931, salarié successivement dans deux entreprises d'imprimerie. La seconde est l'imprimerie Gutenberg, dont les clients principaux sont des organisations syndicales CGT et communistes. Il est mobil

Error: detector_ld("Émile Taranto  oli kanadalainen viulisti, musiikkipedagogi ja säveltäjä. 1   \nTaranto syntyi italialaisille vanhemmille ja aloitti kuusivuotiaana viulunsoiton opiskelun Frantz Jehin-Prumen johdolla. Vuonna 1894 Taranto oli jo Montrealin sinfoniaorkesterin jäsen, joka oli pitänyt myös useita resitaaleja. Kesäkuussa 1903 hän lähti Brysseliin opiskelemaan Eugène Ysaÿen johdolla. Palattuaan Montrealiin hän soitti kotikaupunkinsa sinfoniaorkesterin kanssa Max Bruchin viulukonserton nro 1 huhtikuussa 1904 ja toimi saman orkesterin konserttimestarina 1904–1907. Vuonna 1904 hän myös perusti Mendelssohn-trion. Hän osallistui Beethoven-trion konsertteihin ja kiersi Eurooppaa vuosina 1907 ja 1914. Konserttiartistina toimimisen sivussa hän myös vietti paljon aikaa opettaen; hänen oppilaidensa joukossa olivat Jean Deslauriers, Marthe Lapointe, Annette Lasalle-Leduc ja Lucien Sicotte. 1 \nTaranto sävelsi viulukappaleita, joihin kuuluvat Danse caprice  ja Mouche à feu. Helmikuuss

Error: detector_ld("Ирендѝк (на руски: Ирендык) е планински хребет в централната част на Южен Урал, в близост до изворите на река Урал, в източната част на Република Башкортостан в Русия. Простира се от север на юг на протежение от 120 km. Обособен е в две отделни части: Северен Ирендик с дължина 15 km и ширина 3 – 4 km, простиращ се между най-горните течения на реките Урал на запад и Уй (ляв приток на Тобол) на изток с максимална височина връх Тура Таш 921 m (54°31′37″ с. ш. 59°34′40″ и. д.\ufeff / \ufeff54.526944° с. ш. 59.577778° и. д.54.526944, 59.577778 и Южен Ирендик с дължина около 90 km, простиращ се южно от долината на река голям Кизил (десен приток на Урал) с максимална височина връх Кузгунташ 987 m (53°06′32″ с. ш. 58°22′01″ и. д.\ufeff / \ufeff53.108889° с. ш. 58.366944° и. д.53.108889, 58.366944. Изграден е от порфирити, диабази и варовици. Северната му част е заета от иглолистни гори, а южната – от типчаково-коилови степи.  1 \n")


Error: detector_ld("Луис Ваес де Торес  е испански мореплавател и пътешественик-изследовател.\nПрез 1605 участва като капитан на „Алмиранта“, един от трите кораба в експедицията на Педро Фернандес де Кирос, отправена от испанския крал за търсене на легендарния Южен континент. След като се разделя с Кирос на 11 юни 1606 при неизяснени обстоятелства, изследва Новохебридските о-ви, открива остров Малекула  и се убеждава, че не са част от Южния континент, а група острови.\nС двата отцепили се кораба „Алмиранта“ и „Три влъхви“ Торес се насочва на север и северозапад, пресича Коралово море и на 20 юли 1606 достига до югоизточния край на Нова Гвинея, като открива няколко острова в архипелага Луизиада, в т.ч. островите Росел , Тагула  и Басилаки . Въпреки възприетата по това време традиция да се плава покрай северния бряг на Нова Гвинея, Торес поема курс покрай южния бряг на острова и през септември открива цялото крайбрежие на залива Папуа. Между 9º – 10º ю.ш. и 143º – 144º и.д. открива остро

 87.026929 seconds (991.57 M allocations: 35.321 GiB, 6.10% gc time, 2.17% compilation time)


In [12]:
showtable("Languages.jl"=>wbmk_ls, "LanguageDetect.jl"=>wbmk_ld, header=LANGS)

|                   | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra     | hau    | hbs     | heb     | hin    | hun    | ido | ina | isl | ita    | jpn    | kab | kor    | kur | lat | lit    | mar    | mkd    | msa    | nds | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat | tgl    | tur    | ukr    | vie    | yid    | zho    |
|-------------------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|---------|---------|--------|--------|-----|-----|-----|--------|--------|-----|--------|-----|-----|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|
|      Languages.jl | 99.00% | 98.50% |  99.00% | 99.00% |      - | 92.50% | 88.50% | 96.00% |

|                   | ara    | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng    | epo    | fas    | fin    | fra     | hau    | hbs     | heb     | hin    | hun    | ido | ina | isl | ita    | jpn    | kab | kor    | kur | lat | lit    | mar    | mkd    | msa    | nds | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat | tgl    | tur    | ukr    | vie    | yid    | zho    |
|-------------------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|---------|---------|--------|--------|-----|-----|-----|--------|--------|-----|--------|-----|-----|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|
|      Languages.jl | 99.00% | 98.50% |  99.00% | 99.00% |      - | 92.50% | 88.50% | 96.00% |  96.50% | 99.50% | 96.00% | 98.50% | 98.00% | 100.00% | 99.00% | 100.00% |  99.50% | 91.00% | 93.00% |   - |   - |   - | 98.50% | 99.50% |   - | 89.50% |   - |   - | 94.50% | 95.00% | 98.00% | 99.50% |   - | 94.50% | 95.50% | 90.50% | 94.00% | 81.50% | 97.50% |      - | 98.50% |      - | 88.00% |   - | 97.00% | 92.50% | 93.00% | 74.50% | 98.00% | 96.50% |
| LanguageDetect.jl | 99.50% |      - | 100.00% | 87.50% | 71.50% | 80.50% | 60.00% | 83.50% | 100.00% | 92.00% |      - | 98.50% | 94.00% |  87.50% |      - |   3.50% | 100.00% | 94.50% | 92.00% |   - |   - |   - | 85.00% | 95.50% |   - | 95.00% |   - |   - | 97.00% | 97.00% | 93.50% | 96.00% |   - | 79.00% | 63.00% | 92.00% | 79.50% | 77.00% | 94.50% | 76.00% | 72.50% | 96.50% | 70.00% |   - | 97.00% | 96.50% | 97.00% | 96.50% |      - | 73.50% |
